# DTSA 5511 Final Project# Happywhale - Whale and Dolphin Identification
Identify whales and dolphins by unique characteristic

Kevin Juandi


In this project I am going to work through the Happywhale - Whale and Dolphin Identification Kaggle Competition and build a convolutional nueral network (CNN) to perform image recognition. I chose this project because I am interested in computer vision. I remember that I was interested to participate in this competition when it was live but I didn't have the skills for it. Now that I have learned deep learning methods and computer vision more deeply. I shall attempt to work on this. In this project, I will use a supervised deep learning model to identify our cetacean friends. The benefit of working through a Kaggle competition is the ability to: 

1) learn from others who have submitted open-source code to the competition, and 

2) get immediate feedback from Kaggle as to the performance of my model. 

Here is the general working plan:

1) Download, Import, and Inspect the data

2) Clean the data (if needed)

3) Perform EDA

4) Build an initial CNN, evaluate it against validation

5) Submit best (by validation) CNN predictions to Kaggle

6) Conclusion and Discussion

Before running this notebook, please ensure that you have all the necessary libraries installed. Also please use a kernel with a GPU enabled, otherwise the training time will be overwhelming. Thank you for your time, and I hope you enjoy my project.

## Loading dependanciesmodel.

In [3]:
%matplotlib inline

import numpy as np 
import pandas as pd 

import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
import time
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = Warning)
import sys
import shutil
sys.version

'3.11.7 (main, Dec 15 2023, 18:12:31) [GCC 11.2.0]'

In [5]:
tf.__version__

'2.15.0'

In [6]:
tf.config.list_physical_devices('GPU')

2024-02-24 06:43:30.434156: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:30.581993: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:30.582120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
    if tf.test.is_gpu_available():
        print("GPU is available.")
        !nvidia-smi
    else:
        print("Training on CPU.")

print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available.
Sat Feb 24 06:43:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.07              Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   39C    P0             14W /  115W |     271MiB /   8192MiB |      9%      Default |
|                                        

2024-02-24 06:43:32.760759: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:32.760868: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:32.760919: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:32.919961: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-24 06:43:32.920079: I external/local_xla/xla/stream_executor

Number of replicas: 1


## Importing the data and data cleaning

Here I am using Kaggle's API to download the data for this project. If you haven't already installed the API, just call "!pip install kaggle" in a new cell. Alternatively you can run a notebook through Kaggle's cloud computing resource, which has the benefit of allowing the user free access to GPU runtime. This is a good option if your graphics card is not compatible with TensorFlow. Before downloading the data, please ensure that your working directory is set how you'd like it for this project. You can unzip the data with shutil package. Remember to download API token first from your kaggle user settings page and place it in the right folder.

In [8]:
os.getcwd()

'/mnt/e/Happy-Whale'

In [9]:
!kaggle competitions download -c happy-whale-and-dolphin

happy-whale-and-dolphin.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
#unzip the zip file
shutil.unpack_archive("happy-whale-and-dolphin.zip", "./data", "zip") 
print("Archive file unpacked successfully.")

Archive file unpacked successfully.


In [16]:
#Confirm contents of "./data"
path = "./data/"
os.listdir(path)

['sample_submission.csv', 'test_images', 'train.csv', 'train_images']

In [17]:
# load the train data as well as the sample submission format

train_path = os.path.join(path, 'train_images')
test_path = os.path.join(path, 'test_images')
train_data = pd.read_csv(path + 'train.csv')
sample_sub = pd.read_csv(path + 'sample_submission.csv')

In [18]:
# Preview train data
train_data

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392
...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5ac053677ed1
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9
51031,fffbc5dd642d8c.jpg,beluga,4000b3d7c24e


In [19]:
# Preview sample submission 
sample_sub

,image,predictions
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
...,...,...
27951,fff6ff1989b5cd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
27952,fff8fd932b42cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
27953,fff96371332c16.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
27954,fffc1c4d3eabc7.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...


The training database consists of 51,033 such images, labeled with the species and individual, which we will use to create a classifier that will attempt to classify the species of the animal in each photograph.

In [20]:
train_data.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [21]:
train_data.isnull().sum()

image            0
species          0
individual_id    0
dtype: int64

In [22]:
train_data.describe()

,image,species,individual_id
count,51033,51033,51033
unique,51033,30,15587
top,00021adfb725ed.jpg,bottlenose_dolphin,37c7aba965a5
freq,1,9664,400


Here we see a snapshot of our training data directory. We wee that our data is made up of an image filepath, corresponding to an image contained in the accompanying train folder, as well as data on the species of the whale/dolphin in question, and a code unique to each individual in the database.

What's the breakdown of our "species" feature?

In [23]:
print('Number of unique species: ', train_data['species'].nunique())
species = list(train_data['species'].unique())
species.sort()
print(species)

Number of unique species:  30
['beluga', 'blue_whale', 'bottlenose_dolphin', 'bottlenose_dolpin', 'brydes_whale', 'commersons_dolphin', 'common_dolphin', 'cuviers_beaked_whale', 'dusky_dolphin', 'false_killer_whale', 'fin_whale', 'frasiers_dolphin', 'globis', 'gray_whale', 'humpback_whale', 'kiler_whale', 'killer_whale', 'long_finned_pilot_whale', 'melon_headed_whale', 'minke_whale', 'pantropic_spotted_dolphin', 'pilot_whale', 'pygmy_killer_whale', 'rough_toothed_dolphin', 'sei_whale', 'short_finned_pilot_whale', 'southern_right_whale', 'spinner_dolphin', 'spotted_dolphin', 'white_sided_dolphin']
